In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-10"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13571,2024-12-10,Israel Liga Leumit,14:00,Elitzur Yavne,H. Kfar Saba,1.56,2.21,172.5,1.85,1.85,-4.5,1.93,1.74,K4JUku85,0.641026,0.452489,0.540541,0.540541,0.093514,153.150,31.942467,0.208570,1.8,1.643168,0.912871,185.40,1.632,0.395057,0.242069,35.0,155.404,60.145033,0.387024,2.4,1.341641,0.559017,101.97,1.826,0.772612,0.423117,50.0,90,99,2.06,1.03,150.540,0.000,0.243830,0.000000,0.073215,-3.55,-0.710,-0.788732,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13572,2024-12-10,República Tcheca 1. Liga,14:00,Novy Jicin,Hradec Kralove,11.60,1.02,167.5,1.83,1.83,14.5,1.83,1.83,IymxLWUq,0.086207,0.980392,0.546448,0.546448,0.066599,179.518,87.390086,0.486804,2.4,1.341641,0.559017,132.48,2.234,1.326944,0.593977,17.0,191.484,36.453143,0.190372,3.0,0.000000,0.000000,132.87,1.800,0.318983,0.177213,207.0,96,103,1.38,1.29,146.530,188.974,1.185609,0.000000,0.000000,3.59,0.718,14.763231,0.745449,1.0,0.254551,4.31,0.862,0.023202,0.472569,0.8,0.327431
13573,2024-12-10,República Tcheca Nbl,14:00,Olomoucko,Ostrava,1.78,1.89,170.5,1.86,1.80,-1.5,1.88,1.78,A73iZDyB,0.561798,0.529101,0.537634,0.555556,0.090898,200.664,73.956298,0.368558,0.6,1.341641,2.236068,201.28,2.568,1.083845,0.422058,-19.0,291.342,210.903699,0.723904,1.8,1.643168,0.912871,109.06,3.870,3.466006,0.895609,4.0,74,82,2.72,1.33,237.912,462.100,0.042388,0.023184,0.038640,-2.44,-0.488,-1.598361,0.613824,0.6,-0.013824,1.52,0.304,2.927632,0.257681,0.5,0.242319
13574,2024-12-10,Lituânia Nkl,14:00,Ukmerges Stekas,Delikatesas Joniskis,1.09,5.61,164.5,1.84,1.82,-13.5,1.88,1.78,fiJUgg2b,0.917431,0.178253,0.543478,0.549451,0.095684,177.896,49.911665,0.280567,0.0,0.000000,NaN,179.82,2.420,0.760756,0.314362,-59.0,254.162,110.257366,0.433807,0.0,0.000000,NaN,437.34,3.492,1.426734,0.408572,-100.0,74,74,2.43,5.91,131.376,286.618,0.954066,0.007728,0.038640,-1.77,-0.354,-0.254237,0.585014,0.4,-0.185014,-5.00,-1.000,-4.610000,0.233782,0.0,-0.233782
13575,2024-12-10,Lituânia Nkl,13:00,Vytis VDU,Suduva-Mantinga,1.67,2.02,165.5,1.81,1.85,-3.5,2.04,1.66,WGMezVRk,0.598802,0.495050,0.552486,0.540541,0.093852,155.700,36.798113,0.236340,1.2,1.643168,1.369306,134.40,1.936,0.501328,0.258951,-6.0,126.286,28.862633,0.228550,1.2,1.643168,1.369306,126.42,1.666,0.335604,0.201443,3.0,80,86,1.68,1.47,121.810,119.668,0.134139,0.015456,0.145244,-2.29,-0.458,-1.462882,0.640491,0.5,-0.140491,-3.53,-0.706,-1.444759,0.690810,0.5,-0.190810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13648,2024-12-10,Eua Ncaa,21:00,Syracuse,Albany Great Danes,1.10,7.20,0.0,0.00,0.00,-13.5,1.91,1.80,AZ3hRIFO,0.909091,0.138889,inf,inf,0.047980,431.692,473.287165,1.096354,0.6,1.341641,2.236068,175.36,6.162,6.760057,1.097056,-30.0,247.358,176.996777,0.715549,1.2,1.643168,1.369306,151.70,3.298,2.759505,0.836721,-13.0,64,74,2.74,2.05,108.904,225.612,1.039362,NaN,0.041931,0.23,0.046,2.173913,0.671531,0.8,0.128469,-3.17,-0.634,-9.779180,0.000000,0.0,0.000000
13649,2024-12-10,Eua Ncaa,21:00,Saint Josephs Hawks,Coll Of Charltn,1.34,3.23,150.5,1.80,1.91,-7.5,2.00,1.73,QLYH32cd,0.746269,0.309598,0.555556,0.523560,0.055866,217.706,15

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13578,13:45,Israel Liga Leumit,Ramat Hasharon,Ironi Eilat,1.66,Back Home
13592,14:00,Europa Liga Letônia,Ventspils,Pärnu,1.56,Back Home
13604,20:15,Uruguai Liga Uruguaia,Urupan,Malvin,2.48,Back Home
13647,22:00,Eua Ncaa,South Dakota Coyotes,Wyoming,1.83,Back Home
